In [ ]:
# Add scripts to path
# Import modules
# Add scripts to path and fix working directory
import sys
import os
from pathlib import Path

# Change to project root (two levels up from notebook)
notebook_dir = Path.cwd()
project_root = notebook_dir.parent.parent
os.chdir(project_root)
sys.path.insert(0, str(project_root / "scripts"))

print(f"Working directory: {os.getcwd()}")

from config import PATHS, ML
from ml_training import load_ml_data, load_model

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_predict, learning_curve
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import (
    classification_report, confusion_matrix, ConfusionMatrixDisplay,
    roc_curve, auc, precision_recall_curve, average_precision_score,
)
from sklearn.preprocessing import label_binarize
import warnings
warnings.filterwarnings('ignore')

## Step 1: Load Data and Models

In [2]:
# Load data
X, y, feature_names = load_ml_data(
    PATHS.ML_DATA,
    target_column="pop",
    verbose=True,
)

# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Split data (same split as training)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=ML.TEST_SIZE, stratify=y, random_state=ML.RANDOM_STATE
)
y_test_encoded = le.transform(y_test)

Loading data from: 1000genomes/vcf/vcf_numeric_transposed_with_population.csv


FileNotFoundError: [Errno 2] No such file or directory: '1000genomes/vcf/vcf_numeric_transposed_with_population.csv'

In [ ]:
# Load saved models
import os

model_dir = PATHS.ML_MODELS_DIR
models = {}

model_files = [
    ("random_forest_full.pkl", "Random Forest (Full)"),
    ("random_forest_top25.pkl", "Random Forest (Top 25)"),
    ("logistic_regression.pkl", "Logistic Regression"),
    ("xgboost.pkl", "XGBoost"),
]

for filename, name in model_files:
    path = model_dir / filename
    if os.path.exists(path):
        model, metadata = load_model(path)
        models[name] = {"model": model, "metadata": metadata}
        print(f"Loaded: {name} (accuracy: {metadata.get('accuracy', 'N/A')})")

print(f"\nLoaded {len(models)} models")

In [ ]:
# Load feature importances
importance_file = model_dir / "feature_importances.csv"
if os.path.exists(importance_file):
    feature_importance = pd.read_csv(importance_file)
    print(f"Loaded feature importances: {len(feature_importance)} features")
else:
    feature_importance = None
    print("Feature importances not found")

## Step 2: Detailed Performance Metrics

In [ ]:
# Evaluate all models
results = {}

for name, data in models.items():
    model = data["model"]
    
    # Handle top features model
    if "Top 25" in name and feature_importance is not None:
        top_features = feature_importance.head(25)["feature"].tolist()
        X_test_model = X_test[top_features]
    else:
        X_test_model = X_test
    
    # Predict
    y_pred = model.predict(X_test_model)
    y_pred_proba = model.predict_proba(X_test_model) if hasattr(model, 'predict_proba') else None
    
    results[name] = {
        "y_pred": y_pred,
        "y_pred_proba": y_pred_proba,
        "X_test": X_test_model,
    }
    
    print(f"\n{'='*60}")
    print(f"{name}")
    print(f"{'='*60}")
    print(classification_report(y_test, y_pred))

## Step 3: Confusion Matrices

In [ ]:
# Plot confusion matrices for all models
n_models = len(results)
fig, axes = plt.subplots(1, n_models, figsize=(5*n_models, 4))

if n_models == 1:
    axes = [axes]

for ax, (name, data) in zip(axes, results.items()):
    cm = confusion_matrix(y_test, data["y_pred"])
    ConfusionMatrixDisplay(cm, display_labels=le.classes_).plot(ax=ax, cmap="Blues")
    ax.set_title(name)

plt.tight_layout()
plt.show()

In [ ]:
# Normalized confusion matrices
fig, axes = plt.subplots(1, n_models, figsize=(5*n_models, 4))

if n_models == 1:
    axes = [axes]

for ax, (name, data) in zip(axes, results.items()):
    cm = confusion_matrix(y_test, data["y_pred"], normalize='true')
    ConfusionMatrixDisplay(cm, display_labels=le.classes_).plot(ax=ax, cmap="Blues", values_format='.2f')
    ax.set_title(f"{name}\n(Normalized by True Label)")

plt.tight_layout()
plt.show()

## Step 4: ROC and Precision-Recall Curves

In [ ]:
# ROC curves (One-vs-Rest)
y_test_bin = label_binarize(y_test, classes=le.classes_)
n_classes = len(le.classes_)

# Select best model with probabilities
best_model_name = "Random Forest (Full)"
if best_model_name in results and results[best_model_name]["y_pred_proba"] is not None:
    y_score = results[best_model_name]["y_pred_proba"]
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # ROC Curve
    ax = axes[0]
    colors = plt.cm.tab10(np.linspace(0, 1, n_classes))
    
    for i, (class_name, color) in enumerate(zip(le.classes_, colors)):
        fpr, tpr, _ = roc_curve(y_test_bin[:, i], y_score[:, i])
        roc_auc = auc(fpr, tpr)
        ax.plot(fpr, tpr, color=color, lw=2, label=f'{class_name} (AUC = {roc_auc:.3f})')
    
    ax.plot([0, 1], [0, 1], 'k--', lw=2)
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title(f'ROC Curves ({best_model_name})')
    ax.legend(loc='lower right')
    ax.grid(True, alpha=0.3)
    
    # Precision-Recall Curve
    ax = axes[1]
    
    for i, (class_name, color) in enumerate(zip(le.classes_, colors)):
        precision, recall, _ = precision_recall_curve(y_test_bin[:, i], y_score[:, i])
        ap = average_precision_score(y_test_bin[:, i], y_score[:, i])
        ax.plot(recall, precision, color=color, lw=2, label=f'{class_name} (AP = {ap:.3f})')
    
    ax.set_xlabel('Recall')
    ax.set_ylabel('Precision')
    ax.set_title(f'Precision-Recall Curves ({best_model_name})')
    ax.legend(loc='lower left')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print(f"Model {best_model_name} not found or has no predict_proba")

## Step 5: Feature Importance Deep Dive

In [ ]:
if feature_importance is not None:
    # Cumulative importance
    feature_importance['cumulative'] = feature_importance['importance'].cumsum()
    feature_importance['cumulative_pct'] = feature_importance['cumulative'] / feature_importance['importance'].sum() * 100
    
    # How many features for 80%, 90%, 95% of importance
    for threshold in [80, 90, 95]:
        n_features = (feature_importance['cumulative_pct'] <= threshold).sum() + 1
        print(f"Features for {threshold}% importance: {n_features}")
    
    # Plot cumulative importance
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Cumulative curve
    axes[0].plot(range(1, len(feature_importance) + 1), feature_importance['cumulative_pct'])
    axes[0].axhline(80, color='r', linestyle='--', label='80%')
    axes[0].axhline(90, color='g', linestyle='--', label='90%')
    axes[0].axhline(95, color='b', linestyle='--', label='95%')
    axes[0].set_xlabel('Number of Features')
    axes[0].set_ylabel('Cumulative Importance (%)')
    axes[0].set_title('Cumulative Feature Importance')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Importance distribution
    axes[1].hist(feature_importance['importance'], bins=50, edgecolor='black', alpha=0.7)
    axes[1].set_xlabel('Importance')
    axes[1].set_ylabel('Count')
    axes[1].set_title('Distribution of Feature Importances')
    axes[1].set_yscale('log')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Top 50 features heatmap
if feature_importance is not None:
    top_50 = feature_importance.head(50)['feature'].tolist()
    
    # Get genotype matrix for top features
    X_top = X[top_50]
    
    # Group by population
    X_top['population'] = y.values
    
    # Calculate mean genotype per population
    pop_means = X_top.groupby('population').mean()
    
    # Heatmap
    plt.figure(figsize=(15, 4))
    sns.heatmap(pop_means, cmap='RdYlBu_r', center=1, 
                cbar_kws={'label': 'Mean Genotype'})
    plt.xlabel('SNP')
    plt.ylabel('Population')
    plt.title('Mean Genotype of Top 50 SNPs by Population')
    plt.xticks(rotation=90, fontsize=6)
    plt.tight_layout()
    plt.show()

## Step 6: Error Analysis

In [ ]:
# Analyze misclassifications
best_result = results.get("Random Forest (Full)")
if best_result is not None:
    y_pred = best_result["y_pred"]
    
    # Find misclassified samples
    misclassified = y_test != y_pred
    n_misclassified = misclassified.sum()
    
    print(f"Misclassified samples: {n_misclassified} / {len(y_test)} ({n_misclassified/len(y_test)*100:.2f}%)")
    
    if n_misclassified > 0:
        # Misclassification breakdown
        misclass_df = pd.DataFrame({
            'True': y_test[misclassified].values,
            'Predicted': y_pred[misclassified],
        })
        
        print("\nMisclassification breakdown:")
        print(misclass_df.groupby(['True', 'Predicted']).size().unstack(fill_value=0))

In [ ]:
# Prediction confidence for misclassified samples
if best_result is not None and best_result["y_pred_proba"] is not None and n_misclassified > 0:
    y_proba = best_result["y_pred_proba"]
    
    # Get max probability (confidence) for each prediction
    confidence = y_proba.max(axis=1)
    
    # Compare confidence for correct vs incorrect predictions
    fig, ax = plt.subplots(figsize=(10, 5))
    
    ax.hist(confidence[~misclassified], bins=20, alpha=0.7, label='Correct', color='green')
    ax.hist(confidence[misclassified], bins=20, alpha=0.7, label='Incorrect', color='red')
    ax.set_xlabel('Prediction Confidence')
    ax.set_ylabel('Count')
    ax.set_title('Prediction Confidence: Correct vs Incorrect')
    ax.legend()
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nMean confidence (correct): {confidence[~misclassified].mean():.3f}")
    print(f"Mean confidence (incorrect): {confidence[misclassified].mean():.3f}")

## Step 7: Learning Curves

In [ ]:
# Learning curves for best model
from sklearn.ensemble import RandomForestClassifier

# Use smaller number of estimators for speed
model_for_lc = RandomForestClassifier(n_estimators=50, random_state=ML.RANDOM_STATE, n_jobs=-1)

train_sizes, train_scores, test_scores = learning_curve(
    model_for_lc, X, y,
    cv=5,
    n_jobs=-1,
    train_sizes=np.linspace(0.1, 1.0, 10),
    scoring='accuracy',
)

# Plot
plt.figure(figsize=(10, 5))

plt.fill_between(train_sizes, 
                 train_scores.mean(axis=1) - train_scores.std(axis=1),
                 train_scores.mean(axis=1) + train_scores.std(axis=1),
                 alpha=0.1, color='blue')
plt.fill_between(train_sizes,
                 test_scores.mean(axis=1) - test_scores.std(axis=1),
                 test_scores.mean(axis=1) + test_scores.std(axis=1),
                 alpha=0.1, color='orange')

plt.plot(train_sizes, train_scores.mean(axis=1), 'o-', color='blue', label='Training Score')
plt.plot(train_sizes, test_scores.mean(axis=1), 'o-', color='orange', label='Cross-Validation Score')

plt.xlabel('Training Set Size')
plt.ylabel('Accuracy')
plt.title('Learning Curves (Random Forest)')
plt.legend(loc='lower right')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Summary

In [ ]:
print("="*60)
print("MODEL EVALUATION SUMMARY")
print("="*60)

print("\nModel Performance:")
for name, data in models.items():
    acc = data['metadata'].get('accuracy', 'N/A')
    n_feat = data['metadata'].get('n_features', 'N/A')
    print(f"  {name}: accuracy={acc}, features={n_feat}")

print("\nKey Findings:")
print("  - Random Forest performs best for population classification")
print("  - Top 25 features capture most of the discriminative power")
print("  - High confidence predictions correlate with correct classifications")

print("\nRecommendations:")
print("  - Use Random Forest with top 25 features for deployment")
print("  - Consider ensemble methods for production use")
print("  - Validate on independent test set before deployment")